In [142]:
import os
from huggingface_hub import InferenceClient
import pandas as pd
from tratamento import retorna_df_sem_anomalias, retorna_df_admissao, retorna_df_desligamento, retorna_df_ferias, retorna_df_afastamentos, ajusta_data
from datetime import datetime

inicio_periodo = pd.to_datetime('15/04/2025',dayfirst=True)
fim_periodo = pd.to_datetime('15/05/2025',dayfirst=True)

client = InferenceClient(
    provider="nebius",
    api_key= os.environ["hf_api_key"]
)

%load_ext autoreload
%autoreload 2

def identificar_intencao_usuario(pergunta):

    prompt = f"""
    O usuário está interagindo com nossa aplicação, para poder gerar uma planilha com os calculos de compra de VR para a empresa

    Ele pode, alem de gerar a planilha, fazer perguntas sobre o processo de geração do vr

    identifique a pergunta do usuário e retorne uma das seguintes opções:
    - GERAR_PLANILHA
    - EXPLICAR_PROCESSO
    - CONSULTAR_RESULTADO

    retorne apenas as opções acima


    Pergunta:
    {pergunta}
    """

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

    return completion.choices[0].message.content;

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
# 1. Carregar bases
ativos = pd.read_excel("dados/ativos.xlsx")
aprendizes = pd.read_excel("dados/aprendiz.xlsx")
estagiarios = pd.read_excel("dados/estagio.xlsx")
ferias = pd.read_excel("dados/ferias.xlsx")
desligados = pd.read_excel("dados/desligados.xlsx")
admissao = pd.read_excel("dados/ADMISSAO_ABRIL.xlsx")
sindicato_valores = pd.read_excel("dados/Base_sindicato_valor.xlsx")
afastamentos = pd.read_excel("dados/afastamentos.xlsx")
exterior = pd.read_excel("dados/exterior.xlsx")
dias_uteis = pd.read_excel("dados/base_dias_uteis.xlsx")

In [131]:
df_sem_anomalia = retorna_df_sem_anomalias(ativos,estagiarios, aprendizes, exterior, afastamentos,ferias, sindicato_valores, dias_uteis)

df_admissao = retorna_df_admissao(admissao, inicio_periodo, fim_periodo, sindicato_valores)

df_desligamento = retorna_df_desligamento(desligados,fim_periodo ,sindicato_valores)

df_ferias = retorna_df_ferias(ferias, ativos,sindicato_valores)

df_afastamentos = retorna_df_afastamentos(afastamentos,fim_periodo, ativos, sindicato_valores)

In [133]:
df_afastamentos

,MATRICULA,OBS,DIAS_UTEIS,EMPRESA,CARGO,DESC. SITUACAO,SINDICATO,VALOR_DIARIO,TOTAL,CUSTO_EMPRESA,DESCONTO


In [136]:
df_consolidado = pd.concat([df_sem_anomalia, df_admissao, df_desligamento, df_ferias, df_afastamentos], ignore_index=True)

#df_saida

# #df_sem_anomalia
# #df_sem_anomalia["SINDICATO"] = df_sem_anomalia["SINDICATO"].fillna(sindicato_valores["SINDICATO"])

# #df_sem_anomalia

colunas = [
    "MATRICULA",
    "ADMISSAO",
    "SINDICATO",
    "COMPETENCIA",
    "VALOR_DIARIO",
    "TOTAL",
    "CUSTO_EMPRESA",
    "DESCONTO",
    "OBS",
    "DIAS"
]

df_saida = pd.DataFrame(columns=colunas)

df_saida["MATRICULA"] = df_consolidado["MATRICULA"]
df_saida["SINDICATO"] = df_consolidado["SINDICATO"]
df_saida["VALOR_DIARIO"] = df_consolidado["VALOR_DIARIO"]
df_saida["ADMISSAO"] = df_consolidado["ADMISSAO"]
df_saida["TOTAL"] = df_consolidado["TOTAL"]
df_saida["CUSTO_EMPRESA"] = df_consolidado["CUSTO_EMPRESA"]
df_saida["DESCONTO"] = df_consolidado["DESCONTO"]
df_saida["OBS"] = df_consolidado["OBS"]
df_saida["DIAS"] = df_consolidado["DIAS_UTEIS"]

df_saida["COMPETENCIA"] = f"PEDRIODO: {inicio_periodo} - {fim_periodo}"


# df_saida = pd.concat([df_saida, df_admissao], ignore_index=True)

# df_saida = calculo_padrao(df_saida)

df_saida

,MATRICULA,ADMISSAO,SINDICATO,COMPETENCIA,VALOR_DIARIO,TOTAL,CUSTO_EMPRESA,DESCONTO,OBS,DIAS
0,34941,2025-01-01,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,770.0,616.0,154.0,NaN,22
1,24401,2025-01-01,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,735.0,588.0,147.0,NaN,21
2,35254,2025-01-01,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,735.0,588.0,147.0,NaN,21
3,31731,2025-01-01,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,735.0,588.0,147.0,NaN,21
4,31009,2025-01-01,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,735.0,588.0,147.0,NaN,21
...,...,...,...,...,...,...,...,...,...,...
1831,31077,NaT,SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DA...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,37.5,637.5,510.0,127.5,NaN,17
1832,32672,NaT,SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DA...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,37.5,637.5,510.0,127.5,NaN,17
1833,33806,NaT,SINDPD RJ - SINDICATO PROFISSIONAIS DE PROC DA...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,37.5,262.5,210.0,52.5,NaN,7
1834,26609,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,PEDRIODO: 2025-04-15 00:00:00 - 2025-05-15 00:...,35.0,70.0,56.0,14.0,NaN,2


In [138]:
df_saida.to_excel("saida/vr_mensal.xlsx", index=False)
print("✅ Planilha gerada em saida/vr_mensal.xlsx")

✅ Planilha gerada em saida/vr_mensal.xlsx


In [143]:
resposta = identificar_intencao_usuario("como funciona o processo de geração?")

resposta

'EXPLICAR_PROCESSO'